In [16]:
from sklearn.model_selection import GridSearchCV   #그리드 서치
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [17]:
df = pd.read_csv('https://raw.githubusercontent.com/wikibook/machine-learning/2.0/data/csv/basketball_stat.csv')

train, test = train_test_split(df, test_size=0.2)

In [21]:
def svc_param_selection(X,y,nfolds):
    svm_parameters=[
        {'kernel':['rbf'],
        'gamma':[0.00001,0.0001,0.001,0.01,0.1,1],
        'C':[0.01,0.1,1,10,100,1000]}
    ]

    #사이킷런에서 제공하는 GridSearchCV를 사용해 최적의 파라미터를 구함
    clf = GridSearchCV(SVC(),svm_parameters, cv=10)
    clf.fit(X_train,y_train.values.ravel())
    print(clf.best_params_)

    return clf 

#3점슛과 블로킹 횟수를 학습 데이터로 사용 
X_train = train[['3P','BLK']]

#농구선수 포지션을 예측값으로 사용 
y_train = train[['Pos']]

#최적의 파라미터로 학습된 모델을 clf로 지정
clf = svc_param_selection(X_train, y_train.values.ravel(),10)

{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}


In [ ]:
()#결정 경계선 시각화 
#시각화할 비용 후보들을 저장
C_canditates = []
C_canditates.append(clf.best_params_['C']*0.01)
C_canditates.append(clf.best_params_['C'])
C_canditates.append(clf.best_params_['C']*100)

#시각화 할 감마 후보들을 저장
gamma_candidates = []
gamma_candidates.append(clf.best_params_['gamma']*0.01)
gamma_candidates.append(clf.best_params_['gamma'])
gamma_candidates.append(clf.best_params_['gamma']*100)

#3점슛과 블로킹 횟수로 학습
X = train[['3P', 'BLK']]
#농구선수 포지션을 학습모델의 분류값으로 사용
Y = train['Pos'].tolist()

#시각화를 위한 센터(C)와 슈팅가드(SG)를 숫자로 표현
position = []
for gt in Y :
    if gt == 'C':
        position.append(0)
    else:
        position.append(1)

Classifiers = []

# 파라미터 후보들을조합해서 학습된 모델들을 저장
for C in C_canditates:
    for gamma in gamma_candidates:
        clf = SVC(C=C, gamma=gamma)
        clf.fit(X,Y)
        Classifiers.append((C,gamma,clf))

#각 모델을 시각화
plt. figure(figsize=(18,18))
xx,yy = np.meshgrid(np.linespace(0,4,100), np.linspace(0,4,100))

for (k,(C,gamma,clf)) in enumerate(Classifiers):
    Z = clf.decision_function(np.c_p[xx.ravel(),yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.subplot(len(C_canditates),len(gamma_candidates),k+1)
    plt.title('gamma = 10^%d, C = 10^%d' %(np.log10(gamma), np.log10(C)), size = 'medium')

    plt.pcolormesh(xx,yy, -Z, cmap=plt.cm.RdBu)